# Memory usage estimation

This file extends the memory usage calculations within the playground notebook for specific tasks. 

## Individual radial profiles for halos

We start by estimating the memory use at different stages when calculating the temperature radial profiles for individual halos, taking into account not just FoF particles, but all particles within $2 R_{vir}$. The steps are as follows:

1. Load halo data and restrict it to the 280 halos that have $M > 10^{14}M_\odot$.
2. Load the three fields of all gas particles required for temperature
3. Calculate temperature, discard gas data
4. Load positions of all gas particles (optionally: construct a KD-tree)
5. For a single halo: find particles within sphere of radius $r = 2R_{vir}$ by creating a mask over all particles
6. Calculate radial distance of all particles within the sphere to the halo center
7. Mask particle arrays of temperature and position
8. Calculate the histogram and save it to file

The memory usage will be calculated below.

In [1]:
# SETUP
import h5py
import logging
import logging.config
import matplotlib.pyplot as plt
import numpy as np
import os.path
import sys
from pathlib import Path

import illustris_python as il

# import the helper scripts
module_path = Path(os.getcwd()) / ".." / "src"
sys.path.append(str(module_path.resolve()))
from library.config import logging_config, config
from library import constants

In [2]:
logging_cfg = logging_config.get_logging_config("INFO")
logging.config.dictConfig(logging_cfg)
logger = logging.getLogger("root")
# test setup
logger.info("I am a test log!")

INFO: I am a test log!


### Step 1: Load and restrict halo data

We load the position and mass of 280 halos, resulting in 1 `float32` and 3 `float64` numbers per halo. As such, we can expect to use up:

In [3]:
def estimate_memory_restricted_halos():
    positions = np.zeros((280, 3), dtype=np.float64)
    masses = np.zeros(280, dtype=np.float32)
    size = sys.getsizeof(positions) + sys.getsizeof(masses)
    return size  # in bytes

In [4]:
st1_estimate = estimate_memory_restricted_halos() / 1024
print(f"Estimated size of restricted halos: {st1_estimate} kB")

Estimated size of restricted halos: 7.890625 kB


Due to other data being stored, the actual memory used after this step will be much larger (think configuration variables, the pipeline object, etc.). It is found to actually be:

**Actual memory use after step 1:** 85.9 kB

### Step 2: Load three fields for temperature calculation

We now load the fields `InternalEnergy`, `ElectronAbundance`, and `StarFormationRate` to calculate temperatures.

In [7]:
def get_number_particles():
    cfg = config.get_default_config("TNG300-1")
    f = h5py.File(il.snapshot.snapPath(cfg.base_path, cfg.snap_num, 0), 'r')
    n_part = f["Header"].attrs["NumPart_Total"][0]
    n_part += f["Header"].attrs["NumPart_Total_HighWord"][0] * 2**32
    return n_part


def estimate_memory_gas_data():
    n_parts = get_number_particles()
    return 3 * 32 * n_parts / 8  # in bytes

In [9]:
st2_estimate = estimate_memory_gas_data() / 1024 / 1024 / 1024
print(f"Estimated size of gas data: {st2_estimate} GB")

Estimated size of gas data: 161.54410924762487 GB


This matches the measured memory used well: **actual memory use** is measured to be 161.5 GB.

### Step 3: Calculating temperatures

The peak memory usage during the calculation is difficult to estimate and depends strongly on the method. Using `numpy` and letting it optimize the calculation leads to both the best-case runtime and memory usage of ~175.1 GB of peak memory used for calculation and allocated result array. This leads to a peak memory use of 336.6 GB (measured).

After this step, clean-up reduces the space in memory required to purely the temperature array plus the previouslz allocated memory for halo data and pipeline config. This leads to an estimate of:

In [10]:
st3_estimate = get_number_particles() * 32 / 8  / 1024 / 1024 / 1024
print(f"Estimated memory use after cleaning up: {st3_estimate} GB.")

Estimated memory use after cleaning up: 53.84803641587496 GB.


Again, this matches well with the **measured memory use** after clean-up, which comes out to 53.85 GB.

### Step 4: Loading position data of all gas cells

This is by far the most memory-intensive step, unless the position data is cast to `float32` - it is, by default, loaded as `float64` data. If one casts down to `float32`, then the memory use of the position data is equivalent to that of the gas cell data, i.e. 161.5 GB.

If we use the `float64` data instead, we come out at 323 GB memory use - too much considering the memory-intensive calculations we expect to follow up with. Casting to `float32` seems sensible. 

In addition to this, the temperature data persists, leading to a total memory use after loading position data of:

- `float32` case: 235.45 GB
- `float64` case: 417.05 GB